In [ ]:
for key in np.unique(np.where(loads == 0)[0]):
    print element_keys[key]
    for i in xrange(len(loads[key])):
        if loads[key][i] == 0: print i/12, i%12 + 8
    print '\n\n'

In [ ]:
df = pd.read_csv(os.path.join(raw_transaction_path, '03012016_03312016.csv'))
df = df.loc[df['ElementKey'] == 7266]
df['TransactionDateTime'] = pd.to_datetime(df['TransactionDateTime'])
df['Time'] = df['TransactionDateTime'].dt.time
df['Day'] = df['TransactionDateTime'].dt.weekday
df_ = df.loc[(df['Time'] <= datetime.time(18,0,0)) & (df['Time'] >= datetime.time(16,0,0))]
df_1 = df_.loc[df['Day'] == 1]
df_1

In [ ]:
figure_functions.create_animation(loads, gps_loc, N, P, fig_path, animation_path)

In [ ]:
with open(os.path.join(data_path, 'blockface_locs.p'), 'rb') as f:
    locations = pickle.load(f)

gmap = gmplot.GoogleMapPlotter(47.612676, -122.345028, 15)

for i in np.unique(np.where(loads == 0)[0]):
    gmap.marker(gps_loc[i, 0], gps_loc[i, 1], title=str(element_keys[i]))
gmap.draw(os.path.join(fig_path, 'test.html'))

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
adjacency = np.zeros((len(block_keys), len(block_keys)))
N = len(block_keys)

for i in xrange(N):
    if block_keys[i] in north:
        zone1 = 'north'
    elif block_keys[i] in south:
        zone1 = 'south'
    else:
        print block_keys[i]
    for j in xrange(N):
        if i == j:
            continue
        
        if block_keys[j] in north:
            zone2 = 'north'
        elif block_keys[j] in south:
            zone2 = 'south'
        else:
            print block_keys[j]
        
        if zone1 == zone2:
            adjacency[i, j] = 1
        else:
            pass

In [ ]:
def temporal_day_plots(loads, fig_path, filename='temporal_day_plots.png', show_fig=False):
    """Create subplots containing bar plots of the load for each day of the week.

    This function creates a subplot for each day of week and in each subplot
    is a bar plot with the average load at each hour of the day.

    :param loads: Numpy array where each row is the load data for a block-face
    and each column corresponds to a day of week and hour.
    :param fig_path: Path to save the image to.
    :param filename: Name to save the image as.
    :param show_fig: Bool indicating whether to show the image.
    """

    sns.set()
    sns.set_style("whitegrid")

    fig, ax_list = plt.subplots(nrows=1, ncols=6, figsize=(60, 10), sharey=True)

    # Setting block-faces with negligible load to nan so they are ignored when getting mean.
    with np.errstate(invalid='ignore'):
        loads[loads <= .05] = np.nan

    num_times = loads.shape[1]

    # List of lists, where each inner list contains the indexes for a day.
    days = [[i for i in xrange(j, j+(num_times/6))] for j in xrange(0, num_times, (num_times/6))]

    day_dict = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday',
                5: 'Friday', 6: 'Saturday'}

    day_count = 1
    
    for day in days:
        bins = range(8, (num_times/6) + 8)

        # Getting the mean loads for the hours of the given day and scaling for plot.
        counts = np.nanmean(loads, axis=0)[day] * 100

        #    ax = plt.subplot(1, 6, day_count)
        ax = ax_list[day_count-1]
        ax.set_xticks(np.arange(min(bins), max(bins)+1, 1))
        x_labels = [str(b) + 'AM' if b < 12 else str(b-12) + 'PM' 
                    if b != 12 else str(b) + 'PM' for b in bins]
       # x_labels = [x_labels[i] if i % 2 == 0 else '' for i in xrange(len(x_labels))]
        ax.set_xticklabels(x_labels, fontsize=50, rotation=60, ha='center')
       # ax.set_yticklabels(ax.get_yticklabels(), fontsize=60)
        
        ax.set_title(day_dict[day_count], fontsize=70)
        
      #  plt.setp(ax.get_xticklabels(), fontsize=50, rotation=60)
        plt.setp(ax.get_yticklabels(), fontsize=50)
        
     #   for tick in ax.xaxis.get_majorticklabels():
     #       tick.set_horizontalalignment('left')
        
        ax.bar(bins, counts, width=1, color='red', edgecolor='black', align='edge')

        ax.set_xlim([min(bins), max(bins)+1])
        ax.set_ylim([0, 70])
        
        day_count += 1
    
    fig.text(-.009, .8, r'Occupancy $\%$', ha='center', rotation='vertical', fontsize=70)
    
    sns.reset_orig()

    plt.tight_layout()
    
   # plt.subplots_adjust(right=.5, left=0)

    plt.savefig(os.path.join(fig_path, filename), bbox_inches='tight')

    if show_fig:
        plt.show()

    plt.close()

In [ ]:
key = 1017
df = pd.read_csv(os.path.join(raw_transaction_path, '03012016_03312016.csv'))
df = df.loc[df['ElementKey'] == key]
df = df.sort_values(by='TransactionDateTime')
df['TransactionDateTime'] = pd.to_datetime(df['TransactionDateTime'])
df['Time'] = df['TransactionDateTime'].dt.time
df['Day'] = df['TransactionDateTime'].dt.weekday
df['DurationSeconds'] = df['PaidDuration'].values.astype(int)%60
df['DurationMinutes'] = df['PaidDuration'].values.astype(int)/60
df['DurationHours'] = df['DurationMinutes'].values.astype(int)/60
df['DurationMinutes'] = df['DurationMinutes'] - 60*df['DurationHours']
df['TransactionDateTimeEnd'] = df.apply(lambda row: row['TransactionDateTime'] + datetime.timedelta(hours=row['DurationHours'], minutes=row['DurationMinutes'], seconds=row['DurationSeconds']), axis=1)
df['TimeEnd'] = df['TransactionDateTimeEnd'].dt.time
df['Date'] = df['TransactionDateTime'].dt.date

df_ = df.loc[(df['Time'] <= datetime.time(11,0,0)) & (df['Time'] >= datetime.time(9,0,0))]
df_1 = df_.loc[df['Date'] == datetime.date(2016,3,3)]
for i in xrange(1, 59):
    print i, len(df_1.loc[(df_1['Time'] <= datetime.time(9,i,0)) & (df_1['TimeEnd'] >= datetime.time(9,i+1,0))])/9.

In [ ]:
pd.set_option('display.max_columns', None)  

In [ ]:
time = 58
run_all.run_figures(element_keys, loads, gps_loc, N, P, data_path, fig_path, animation_path, time)

In [ ]:
hour_count = 0

for out_time in xrange(12):
    curr_time = []
    for time in xrange(out_time, 60, 12):
        curr_time.append(time)
        
    figure_functions.mixture_plot(loads=loads, gps_loc=gps_loc, times=curr_time,
                                  fig_path=fig_path, num_comps=num_comps, 
                                  filename='test_mixture_same_hour_no_sat' + str(hour_count) +  '.png', 
                                  shape=(1, 5), show_fig=True)

    hour_count += 1

In [ ]:
curr_time = [14, 15, 16, 17, 18]

figure_functions.mixture_plot(loads=loads, gps_loc=gps_loc, times=curr_time,
                              fig_path=fig_path, num_comps=num_comps, 
                              filename='test_mixture_same_day.png', 
                              shape=(1, 5), show_fig=True)


In [ ]:
array([ 38.74,  28.  ,  11.96,  44.03])
dark blue, green, pink, light blue
30.683448014810018